In [10]:
import torch
import numpy as np
import scipy.io
import os
import logging
import sys
import time
from util import getDefaultHyperparams, extractBatch, trainModel, validateModel, testModel
from SingleDataloader import DataProcessing, CustomBatchSampler, TestBatchSampler
from DayDataloaders import create_Dataloaders, DayInfiniteIterators
from PrepareData import PrepareData
from torch.utils.data import DataLoader
from network import Net, RSNNet
from SequenceLoss import SequenceLoss

In [11]:
hyperparams = getDefaultHyperparams()
hyperparams['batch_size'] = 20
hyperparams['train_val_timeSteps'] = 1200

Please set: /n hyperparams['outputDir']


DATASET PREPARATION

In [14]:
prepared_data = PrepareData(hyperparams)

# loading the training dataset and creating a DataLoader
Train_dataset = DataProcessing(hyperparams, prepared_data, mode='training')
trainDayBatch_Sampler = CustomBatchSampler(Train_dataset.getDaysIdx(), hyperparams['batch_size'])
train_loader = DataLoader(Train_dataset, batch_sampler = trainDayBatch_Sampler , num_workers=2)
print('Training data loaded')
# 


# loading the validation dataset and creating a DataLoader
Val_dataset = DataProcessing(hyperparams, prepared_data, mode='validation')
valDayBatch_Sampler = CustomBatchSampler(Val_dataset.getDaysIdx(), hyperparams['batch_size'])
val_loader = DataLoader(Val_dataset, batch_sampler = valDayBatch_Sampler, num_workers=2)
print('Validation data loaded')

# loading the testing dataset and creating a DataLoader for each day
Test_finite_loader = create_Dataloaders(hyperparams, days=np.arange(10), mode='testing')
test_loaders = Test_finite_loader.getDataloaders()
viable_test_days = Test_finite_loader.getViableDays()
print('Testing data loaded')

Training data loaded
Validation data loaded
Testing data loaded


MODEL CREATION

In [15]:
# Device selection
device = torch.device('cpu')
if torch.cuda.is_available():
    print('GPU available')
    device = torch.device('cuda')

# Model creation
model = Net(hyperparams)
model.to(device)

# Loss function
criterion = SequenceLoss(hyperparams)

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=hyperparams['learning_rate'], 
                              betas= (0.9, 0.999), eps=1e-08, 
                              weight_decay=hyperparams['weight_decay'], amsgrad=False)

GPU available


TRAINING AND VALIDATION

In [16]:
# Start timer
training_start = time.time()
epochs = hyperparams['epochs']

for epoch in range(epochs):
    # Start epoch timer
    epoch_start = time.time()
    print(f"\nEpoch {epoch+1}")

    # Training epoch
    train_loss = trainModel(model, train_loader , optimizer, criterion, hyperparams, device)
    # Validation epoch
    val_loss, val_acc = validateModel(model, val_loader, criterion, hyperparams, device)

    epoch_end = time.time()
    print(f"Epoch time: {epoch_end - epoch_start:.2f} s")
    # Print results of the epoch
    print(f"Train loss: {train_loss:.4f} | Validation loss: {val_loss:.4f} | Validation accuracy: {val_acc:.4f}")

training_end = time.time()
print('Training time: ', training_end - training_start)

# Save the model
torch.save(model.state_dict(), 'model.pth')


Epoch 1


TypeError: trainModel() takes 5 positional arguments but 6 were given

TESTING 

In [ ]:
# Start testing
test_start = time.time()
testModel(model, test_loaders, viable_test_days,hyperparams, device)
test_end = time.time()